## 練習時間
將你的結果存成 csv, 上傳你的第一份 Kaggle 成績

Hints: https://stackoverflow.com/questions/16923281/pandas-writing-dataframe-to-csv-file

# [作業目標]
- 檢視並重新執行腳本, 試著將執行結果存檔並提交到 Kaggle 上

# [作業重點]
- 執行後, 將所得結果, 存成可以提交的 csv 檔 (In[10]) (Hint : 使用 pandas.DataFreame.to_csv)

# [提醒]
- 執行後, 將所得結果, 存成可以提交的 csv 檔 (In[10]) (Hint : 使用 pandas.DataFreame.to_csv)

In [1]:
# Import 需要的套件
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = 'C:/Users/rich chang/Documents/GitHub/3rd-ML100Days/data/'
training_set = pd.read_csv(os.path.join(dir_data, 'application_train.csv'))
testing_set = pd.read_csv(os.path.join(dir_data, 'application_test.csv'))
train_labels = training_set['TARGET']


### 仿造之前做過的處理

In [2]:
# 種類 2 種以下的類別型欄位轉標籤編碼 (Label Encoding)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0  # 計數器紀錄有多少個 columns 被標籤編碼過

# 檢查每一個 column，如果只有兩種值的類別型欄位，就做 Label Encoder
for col in training_set:
    if training_set[col].dtype == 'object':
        if len(list(training_set[col].unique())) <= 2:
            le.fit(training_set[col])
            training_set[col] = le.transform(training_set[col])
            testing_set[col] = le.transform(testing_set[col])
            le_count += 1

# 標籤編碼 (2種類別) 欄位轉 One Hot Encoding            
training_set = pd.get_dummies(training_set)
testing_set = pd.get_dummies(testing_set)


In [3]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
training_set['DAYS_EMPLOYED_ANOM'] = training_set["DAYS_EMPLOYED"] == 365243
training_set['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
testing_set['DAYS_EMPLOYED_ANOM'] = testing_set["DAYS_EMPLOYED"] == 365243
testing_set["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
training_set['DAYS_BIRTH'] = abs(training_set['DAYS_BIRTH'])
testing_set['DAYS_BIRTH'] = abs(testing_set['DAYS_BIRTH'])


### 做好前處理
開始擬合模型之前，我們要確保 training & testing data 的欄位數量一致，原因是因為 One hot encoding 會製造多的欄位，有些類別出現在 training data 而沒有出現 testing data 中，我們就要把這些多餘的欄位去除

In [4]:
# 調整欄位數, 移除出現在 training data 而沒有出現 testing data 中的欄位
training_set, testing_set = training_set.align(testing_set, join='inner', axis=1)


In [5]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# 特徵欄位清單
features = list(training_set.columns)

# 填補器 : 設定缺失值補中位數
imputer = Imputer(strategy='median')

# 填補器載入各欄中位數，並回填 train, test 資料中的空缺值
imputer.fit(training_set)
training_set_ = imputer.transform(training_set)
testing_set_ = imputer.transform(testing_set)

# 縮放器 : 設定特徵縮放到 0~1 區間
scaler = MinMaxScaler(feature_range=(0, 1))

# 縮放器載入 train 的上下限, 對 train, test 進行縮放轉換
scaler.fit(training_set_)
training_set_ = scaler.transform(training_set_)
testing_set_ = scaler.transform(testing_set_)

print('Training data shape: ', training_set_.shape)
print('Testing data shape: ', testing_set_.shape)


C:\Users\rich chang\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Training data shape:  (307511, 240)
Testing data shape:  (48744, 240)


### Fit the model

In [6]:
from sklearn.linear_model import LogisticRegression

# 設定模型與模型參數
log_reg = LogisticRegression(C = 0.0001)

# 使用 Train 資料訓練模型
log_reg.fit(training_set_, train_labels)


C:\Users\rich chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

模型 fit 好以後，就可以用來預測 testing data 中的客戶違約遲繳貸款的機率咯! (記得要用 predict_proba 才會輸出機率)

In [7]:
# 用模型預測結果
# 請注意羅吉斯迴歸是分類預測 (會輸出 0 的機率, 與 1 的機率), 而我們只需要留下 1 的機率這排
log_reg_pred = log_reg.predict_proba(testing_set_)[:, 1]


### 儲存預測結果

In [8]:
# 計算提交結果
submit = testing_set[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred
submit.head()


C:\Users\rich chang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.065051
1,100005,0.126401
2,100013,0.081239
3,100028,0.061509
4,100038,0.128308


In [9]:
# transfer to CSV file
submit.to_csv('Home_credit_default_risk_predict.csv', index = False)
